In [10]:
import os
import cv2
import numpy as np
import open3d as o3d

In [11]:
annot_folder_path = "C:\\Users\\dimaj\\Desktop\\full_icl\\output"
depth_folder_path = "C:\\Users\\dimaj\\Desktop\\living_room_traj0_frei_png.tar\\living_room_traj0_frei_png\\depth"

In [12]:
class CameraIntrinsics:
    def __init__(self, width, height, fx, fy, cx, cy, factor):
        self.width = width
        self.height = height
        self.fx = fx
        self.fy = fy
        self.cx = cx
        self.cy = cy
        self.factor = factor
        self.open3dIntrinsics = o3d.camera.PinholeCameraIntrinsic(
            width=width,
            height=height,
            fx=fx,  # X-axis focal length
            fy=fy,  # Y-axis focal length
            cx=cx,  # X-axis principle point
            cy=cy  # Y-axis principle point
        )

In [13]:
def normalize_color_arr(color_arr):
    return color_arr / 255

In [14]:
def depth_to_pcd_custom(depth_image, camera_intrinsics: CameraIntrinsics, initial_pcd_transform):
    points = np.zeros((camera_intrinsics.width * camera_intrinsics.height, 3))

    column_indices = np.tile(np.arange(camera_intrinsics.width), (camera_intrinsics.height, 1)).flatten()
    row_indices = np.transpose(np.tile(np.arange(camera_intrinsics.height), (camera_intrinsics.width, 1))).flatten()

    points[:, 2] = depth_image.flatten() / camera_intrinsics.factor
    points[:, 0] = (column_indices - camera_intrinsics.cx) * points[:, 2] / camera_intrinsics.fx
    points[:, 1] = (row_indices - camera_intrinsics.cy) * points[:, 2] / camera_intrinsics.fy

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.transform(initial_pcd_transform)

    return pcd


In [15]:
class TumIclLoader:
  def __init__(self):
    self.config = self.provide_config()

    depth_path = depth_folder_path

    depth_filenames = self.provide_filenames(depth_path)

    self.depth_images = [os.path.join(depth_path, filename) for filename in depth_filenames]

  def read_depth_image(self, frame_num) -> np.array:
      depth_frame_path = self.depth_images[frame_num]
      return cv2.imread(depth_frame_path, cv2.IMREAD_ANYDEPTH)

  def read_pcd(self, frame_num):
      depth_image = self.read_depth_image(frame_num)
      cam_intrinsics = self.config.get_cam_intrinsic(depth_image.shape)
      initial_pcd_transform = self.config.get_initial_pcd_transform()

      pcd = depth_to_pcd_custom(depth_image, cam_intrinsics, initial_pcd_transform)

      return pcd

  def provide_filenames(self, depth_path):
      depth_filenames = os.listdir(depth_path)
      normalized_depth_filenames = TumIclLoader.__normalize_filenames(depth_filenames)
      normalized_depth_filenames.sort()

      return normalized_depth_filenames

  def provide_config(self):
      return self.TumIclCameraConfig()

  @staticmethod
  def __normalize_filenames(filenames):
      max_filename_len = max([len(filename[:-4]) for filename in filenames])
      normalized_filenames = [
          "0" * (max_filename_len - len(filename[:-4])) + filename for filename in filenames
      ]

      return normalized_filenames

  class TumIclCameraConfig:
      def get_cam_intrinsic(self, image_shape=(480, 640)) -> CameraIntrinsics:
          # Taken from https://www.doc.ic.ac.uk/~ahanda/VaFRIC/codes.html
          return CameraIntrinsics(
              width=image_shape[1],
              height=image_shape[0],
              fx=481.20,  # X-axis focal length
              fy=-480.00,  # Y-axis focal length
              cx=319.50,  # X-axis principle point
              cy=239.50,  # Y-axis principle point
              factor=5000  # for the 16-bit PNG files
          )

      def get_initial_pcd_transform(self):
          return [[-1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]

In [16]:
class DiAnnotation:
    def __init__(self):
        self.path = annot_folder_path
        annotations = [os.path.join(self.path, filename) for filename in os.listdir(self.path)]
        self.annotations = sorted(annotations, key=lambda x: int(x.split("\\")[-1].split(".")[0]))

    def annotate(self, pcd: o3d.geometry.PointCloud, frame_num: int) -> o3d.geometry.PointCloud:
        annotation = self.annotations[frame_num]
        annotation_rgb = cv2.imread(annotation)
        colors = normalize_color_arr(annotation_rgb.reshape((annotation_rgb.shape[0] * annotation_rgb.shape[1], 3)))

        pcd.colors = o3d.utility.Vector3dVector(colors)

        return pcd

In [ ]:
loader = TumIclLoader()
frame_num = 0
pcd = loader.read_pcd(frame_num)
annot = DiAnnotation()
res_pcd = annot.annotate(pcd, frame_num)
o3d.visualization.draw_geometries([res_pcd])
